In [1]:
appliances = {
        0: {'app': 'kettle', 'power': 2000},
        # 1: {'app': 'toaster', 'power': 1500},
        # 2: {'app': 'dish washer', 'power': 2200},
        # 3: {'app': 'microwave', 'power':1500}
        # 4: {'app': 'hair dryer', 'power': 1000},
        # 5: {'app': 'audio amplifier', 'power': 20},
    }

#air dryer nao tem na casa 2?
#tirar hair dryer e audio amplifier!!!
#falta testar onde fica o dropout da ann!!

deep_learning_technique = ['ANN']
# deep_learning_technique = ['ANN', 'CNN', 'GRU']

homes = [1,2,5]

#esse dict soh serve pra dar load na rede
app_to_image_size = {
    'kettle': 48,
    'toaster': 64,
    'audio amplifier': 0,
    'dish washer': 166,
    'hair dryer': 27,
    'microwave': 42
}

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import nilmtk
from nilmtk import DataSet, MeterGroup
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import seaborn as sns
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

sns.set()
plt.style.use('ggplot')
rcParams['figure.figsize'] = (20, 8)
caminho_ukdale = '../ukdale.h5'

# Lendo Dataset do Ukdale
ukdale = DataSet(caminho_ukdale)

In [3]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"

device = torch.device(dev)

In [4]:
class MyDataset(Dataset):
    def __init__(self, data, image_size, target_cols):
        self.data = torch.from_numpy(data).to(device)
        self.image_size = image_size
        self.target_cols = target_cols
        self.shape = self.__getshape__()
        self.size = self.__getsize__()
 
    def __getitem__(self, index):
        x = self.data[index:index+self.image_size, 0:self.target_cols]
        y = self.data[index:index+self.image_size, self.target_cols]
        return x, y
 
    def __len__(self):
        return len(self.data)
    
    def __getshape__(self):
        return (self.__len__(), *self.__getitem__(0)[0].shape)
    
    def __getsize__(self):
        return (self.__len__())

In [5]:
def create_dataset(ukdale, appliance, power, house):
    # Lendo eletrodomesticos da Casa House
    elec = ukdale.buildings[house].elec

    #Criando dataset do mains
    main_df = next(elec.mains().load())
    # print(main_df.head())
    main_df.columns = [' '.join(col).strip() for col in main_df.columns.values]
    main_df = main_df.rename(columns={'power apparent': 'pwr_app_mains', 'power active': 'pwr_act_mains', 'voltage': 'voltage_mains'})
    main_df = main_df.sort_index()
    
    appliance_df = next(elec[appliance].load())
    appliance_df.columns = ['pwr_act_' + appliance]
    appliance_df = appliance_df.sort_index()
    
    #-------------------Upsample do main_df--------------------------#
    appliance_df = appliance_df[appliance_df.index >= main_df.index[0]]

    main_df = main_df.resample('6S', offset='5s').mean().interpolate()
    main_df = main_df[main_df.index >= main_df.index[1]]
    main_df = main_df[main_df.index <= appliance_df.index[-1]]

    #------------------Resampling appliance_df-----------------------#
    appliance_df = appliance_df.resample('6S', offset='5s').mean().interpolate()

    #-------------------Merging dataframes---------------------------#
    merged_df = main_df.merge(appliance_df, left_index=True, right_index=True)
    merged_df['time'] = merged_df.index
    merged_df['time'] = merged_df['time'].apply(lambda x: x.tz_localize(None))
    merged_df.index = merged_df['time']
    merged_df['dia_semana'] = merged_df.time.dt.dayofweek
    merged_df['mes'] = merged_df.time.dt.month
    merged_df['hora'] = merged_df.time.dt.hour
    merged_df = merged_df.drop(columns=['time'])

    #---------------Criando coluna classificatoria-------------------#
    merged_df['ligado'] = merged_df['pwr_act_' + appliance] > power
    
    return merged_df

In [6]:
def separate_training_from_test(df):
    #Fazendo a divisao entre treino e teste de forma que 1 milhao de pontos sejam utilizados para treinar
    train_test_divide = df.index[int(0.8*len(df))]

    print("inicio treino: " + str(df.index[0]))
    print("final treino: " + str(train_test_divide))
    print("final teste: " + str(df.index[-1]))
    
    train = df[df.index <= train_test_divide]
    test = df[df.index > train_test_divide]
    
    return train,test

In [7]:
def calculate_batch_size(df):
    df["ligado"] = df["ligado"].astype(int)
    df_ligado = df[df['ligado'] == True][['ligado']].reset_index()
    df_ligado['diferenca'] = df_ligado['time'].diff()
    df_ligado[df_ligado['diferenca'] > '00:00:30']
    
    
    # add a column saying if a row belongs to the same state as the one before it
    df_ligado["is_first"] = df_ligado['diferenca'] > "00:00:06"

    # the cumulative sum - each "clump" gets its own integer id
    df_ligado["value_group"] = df_ligado["is_first"].cumsum()

    # get the rows corresponding to states beginning
    start = df_ligado.groupby("value_group", as_index=False).nth(0)
    # get the rows corresponding to states ending
    end = df_ligado.groupby("value_group", as_index=False).nth(-1)

    start_end = pd.DataFrame(
        {
            "start": start.index,
            # add freq to get when the state ended
            "end": end.index,
        }
    )
    # convert timedeltas to seconds (float)
    start_end["duration"] = (
        (start_end["end"] - start_end["start"]).apply(np.int64)
    )

    biggest_window = max(start_end.duration)
    return biggest_window

In [8]:
def create_sequential_dataset(df,  image_size, batch_size, target_cols):
    df = df[['voltage_mains', 'pwr_app_mains', 'pwr_act_mains', 'dia_semana', 'hora', 'ligado']]
    df_scaled = pd.DataFrame(np.append(
        preprocessing.scale(df[['voltage_mains', 'pwr_app_mains', 'pwr_act_mains', 'dia_semana', 'hora']]), df[['ligado']].values, axis=1),
                                index=df.index, columns=df.columns)
    dataset = MyDataset(df_scaled.values, image_size, target_cols)
    dfloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size)
    return dfloader

In [9]:
def choose_net(dp_tec,  image_size, batch_size):
    n=5
    if dp_tec == 'ANN':
        class Net(nn.Module):    
            def __init__(self):
                super(Net, self).__init__()
                self.hd1 = nn.Linear(image_size*n, 500*n)
                self.hd2 = nn.Linear(500*n, 500*n)
                self.hd3 = nn.Linear(500*n, 500*n)
                self.hd4 = nn.Linear(500*n, image_size)
                self.bn = nn.BatchNorm1d(500*n)
                self.dropout = nn.Dropout(p=0.5)

            def forward(self, x):
                x = F.relu(self.hd1(x.reshape(x.size(0), -1)))
                x = F.relu(self.bn(self.hd2(x)))
        #         x = self.bn(x) aqui foi o melhor
                # x = self.dropout(x) tava aqui
                x = F.relu(self.hd3(x))
                x = self.hd4(x)
                x = self.bn(x)
                x = self.dropout(x)
                return x

        def init_weights(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)
        net = Net()

    if dp_tec == 'CNN':            
        class Net(nn.Module):    
            def __init__(self):
                super(Net, self).__init__()
                self.pool1 = nn.MaxPool1d(5,1)
                self.cv1 = nn.Conv1d(n, n*8, 3)
                self.cv2 = nn.Conv1d(n*8, n*8, 3)
                self.cv3 = nn.Conv1d(n*8, 16*n, 3)
                self.cv4 = nn.Conv1d(16*n, 16*n, 3)
                self.cv5 = nn.Conv1d(16*n, 32*n, 3)       
                self.cv6 = nn.Conv1d(32*n, 32*n, 3)
                self.cv7 = nn.Conv1d(32*n, 64*n, 3)
                self.cv8 = nn.Conv1d(64*n, 64*n, 3)
                self.fc1 = nn.Linear(64*n*(image_size - 40), 300)
                self.fc2 = nn.Linear(300, image_size)
                self.bn = nn.BatchNorm1d(64*n)
                self.dropout = nn.Dropout(p=0.75)
            def forward(self, x):
                x = F.relu(self.cv2(F.relu(self.cv2(F.relu(self.cv1(x))))))
                x = self.pool1(x)
                x = F.relu(self.cv4(F.relu(self.cv4(F.relu(self.cv3(x))))))
                x = self.pool1(x)
                x = F.relu(self.cv6(F.relu(self.cv6(F.relu(self.cv5(x))))))
                x = self.pool1(x)
                x = F.relu(self.cv8(F.relu(self.cv8(F.relu(self.cv7(x))))))
                x = self.bn(x)
                x = self.pool1(x)
                x = F.relu(self.fc1(x.reshape(x.size(0), -1)))
                x = self.dropout(x)
                x = self.fc2(x)
                return x
        def init_weights(m):
            if type(m) == nn.Conv1d or type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01) 
        net = Net()


    if dp_tec == 'GRU':
        class EncoderRNN(nn.Module):
            def __init__(self):
                super(EncoderRNN, self).__init__()
                self.cv1 = nn.Conv1d(n, n*8, 3)
                self.gru1 = nn.GRU(40, 128, num_layers = 2, bidirectional = True, batch_first = True)
                self.fc1 = nn.Linear(256, 100)
                self.fc2 = nn.Linear(100, image_size)
                self.bn = nn.BatchNorm1d(image_size-2)
                self.dropout = nn.Dropout(p=0.6)
            def forward(self, x, hidden):
                x = torch.transpose(x, 1, 2)
                x = F.relu(self.cv1(x))
                x = torch.transpose(x, 1, 2)        
                x, hidden = self.gru1(x, hidden)
                x = self.bn(x)
                x = F.relu(self.fc1(F.relu(x[:,-1])))
                x = self.dropout(x)
                x = self.fc2(x)
                return x, hidden
            def initHidden(self):
                return torch.zeros(4, batch_size, 128, device=device)
        def init_weights(m):
            if type(m) == nn.Conv1d or type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)
            if type(m) == nn.GRU:
                for layer_p in m._all_weights:
                    for p in layer_p:
                        if 'weight' in p:
                            #print(p, m.__getattr__(p))
                            torch.nn.init.xavier_uniform_(m.__getattr__(p))
                        if 'bias' in p:
                            m.__getattr__(p).data.fill_(0.01)
        def repackage_hidden(h):
            """Wraps hidden states in new Tensors, to detach them from their history."""
            if isinstance(h, torch.Tensor):
                return h.detach()
            else:
                return tuple(repackage_hidden(v) for v in h)
        net = EncoderRNN()

    #inicializacao
    net = net.to(device)
    net.apply(init_weights)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    net.zero_grad()
    
    return net, criterion, optimizer

In [10]:
def train_net(net, trainloader, optimizer, criterion, appliance, dp_tec, image_size):
    comeco = datetime.datetime.now()
    print('###########################')
    print('Comeco do treino para ', appliance, ' utilizando a rede ', dp_tec, ' inicio as ', comeco)
    if dp_tec == 'GRU':
        net.train()
    # for epoch in range(1):
    for epoch in range(5):  # loop over the dataset multiple times
        hidden = False
        real_i = 0
        running_loss = 0.0
        if dp_tec == 'GRU':
            hidden = net.initHidden()
        for i, data in enumerate(trainloader, 0):
            if i > len(trainloader)-image_size:
                break
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            labels = labels.to(device)
            if torch.max(labels).item() == 0:
                continue     
            optimizer.zero_grad()

            if dp_tec == 'CNN':
                inputs = torch.transpose(inputs,1,2)
            if dp_tec == 'GRU':
                    hidden = hidden.data
                    net.zero_grad()
                    outputs, hidden = net(inputs.float(), hidden)
            else:
                # forward + backward + optimizepadded input size per channel: 
                outputs = net(inputs.float())

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            real_i += 1

            # print statistics
            running_loss += loss.item()
            if real_i % 100 == 99:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, real_i + 1, running_loss / 100))
                running_loss = 0.0

    duracao = datetime.datetime.now() - comeco
    print('Finished Training ', dp_tec, ' para ', appliance, ' com duracao de ', duracao)
    print('terminou as ', datetime.datetime.now())
    net.eval()
    return net, hidden

In [11]:
def save_net(net, appliance, dp_tec):  
    onde_salvar_rede = 'BCEwithlogits_'
    nome_appliance = appliance.replace(' ', '_')
    # onde_salvar_rede = onde_salvar_rede + dp_tec + '_' + nome_appliance + '_' + 'wo_timestamp.pt'
    onde_salvar_rede = onde_salvar_rede + dp_tec + '_' + nome_appliance + '_' + 'wo_timestamp_bn_dropout.pt'

    
    torch.save(net.state_dict(), onde_salvar_rede)
    print('rede salva com sucesso em:')
    print(onde_salvar_rede)

In [12]:
def load_net(path, dp_tec, appliance, image_size, batch_size):
    image_size = app_to_image_size[appliance]
    net,_,_ = choose_net(dp_tec, image_size, batch_size)
    # net = torch.load(path, map_location=device)
    net.load_state_dict(torch.load(path))
    print(net.eval())
    return net

In [13]:
def print_start_test(appliance, dp_tec, house, is_test=False):
    comeco = datetime.datetime.now()
    print("-----------------------------")
    if is_test == True:
        print('comeco do teste para ', appliance, 'utilizando ', dp_tec, 'foi as', comeco)
    else:
        print('comeco da validacao para casa', str(house), 'do ', appliance, 'utilizando ', dp_tec, 'foi as', comeco)


def test_net(net, dfloader, dp_tec, image_size, hidden = False):
    comeco_teste = datetime.datetime.now()
    print(net.eval())
    array_results = []

    for i, data in enumerate(dfloader, 0):

        if i > len(dfloader)-image_size:
            break

        if i % 10000 == 9999:
            print(i + 1)
            print(datetime.datetime.now() - comeco_teste)

        inputs, _ = data
        if dp_tec == 'CNN':
            inputs = torch.transpose(inputs,1,2)
        if dp_tec == 'GRU':
            hidden = hidden.data
            out, hidden = net(inputs.float(), hidden)
        else:
            out = net(inputs.float())

        for element in out:
            array_results.append(element.detach().cpu().numpy())

    duracao = datetime.datetime.now() - comeco_teste
    print('duracao do teste:', duracao)
    print("-----------------------------")
    return array_results

def out_into_df(df, out, image_size):
    df['saidas'] = 0.0
    array = df.loc[:, 'saidas'].values

    for i, element in enumerate(out):
        array[i:i+image_size] += element
    # array=array/image_size    
    df.loc[:, ('saidas')] = array/image_size
    return df

def save_df_into_csv(df, dp_tec, appliance, house, is_test=False):
    if is_test == True:
        nome_arquivo = 'saida_teste_'

    else:
        nome_arquivo = 'saida_casa_' + str(house) + '_'

    nome_appliance = appliance.replace(' ', '_')
    # nome_arquivo = nome_arquivo + dp_tec + '_' + nome_appliance + '_wo_timestamp.csv'
    nome_arquivo = nome_arquivo + dp_tec + '_' + nome_appliance + 'wo_timestamp_bn_dropout.csv'

    

    df.to_csv(nome_arquivo, sep=',', header=True, index=True)
    return True

# def apply_threshold(df, dp_tec):
#     df = df.copy()
#     df.loc[:, 'ligado'] = df['ligado'].astype(int)
#     df.loc[:,'saidas'] = df['saidas'].apply(lambda x: x if x > 0 else 0)
#     threshold = try_thresholds(df)
#     df.loc[:,'saidas'] = df['saidas'].apply(lambda x: 1 if x > threshold else 0)
#     return df

def open_csv_as_df(dp_tec, appliance, house, is_test=False):
    if is_test == True:
        nome_arquivo = 'saida_teste_'
        # nome_arquivo = 'saida_teste_dropout2_' #TODO tirar dropout
    else:
        nome_arquivo = 'saida_casa_' + str(house) + '_'
        # nome_arquivo = 'saida_dropout_2_casa_' + str(house) + '_' #TODO tirar dropout
    nome_appliance = appliance.replace(' ', '_')
    nome_arquivo = 'wo_timestamp/'+nome_arquivo + dp_tec + '_' + nome_appliance + '_wo_timestamp.csv'
    df = pd.read_csv(nome_arquivo, sep=',')
    return df

In [15]:
comeco_total = datetime.datetime.now()
print('comecou as ', comeco_total)

for i in appliances:
    comeco_aparelho = datetime.datetime.now()
    
    ukdale = DataSet(caminho_ukdale)

    #Escolha se deseja escolher uma janela de tempo e o período de inicio e fim:    
    set_timewindow = True
    startDate = '2013-03-01'
    endDate = '2013-09-01'
    
    #Escolha do eletrodomestico:
    appliance = appliances[i]['app']
    power = appliances[i]['power']
    
    if set_timewindow:
        ukdale.set_window(start=startDate, end=endDate)
    
    print('Comeco do aparelho ', appliance , ' foi as ', comeco_aparelho)
    #create Dataset
    df = create_dataset(ukdale, appliance, power, 1)
    
    #Desagregacao
    train,test = separate_training_from_test(df)

    #Batch Size
    biggest_window = calculate_batch_size(train)

    #Criando Dataset sequencial
    image_size = biggest_window + 2
    if image_size < 41:
        image_size = 42
    batch_size = 64
    target_cols = 5
    
    trainloader = create_sequential_dataset(train, image_size, batch_size, target_cols)
    
    #pro futuro:
    print(appliance, ' image_size =', image_size)
    
    ####running all for all appliances#####
    for dp_tec in deep_learning_technique:
        
        #training
        torch.cuda.empty_cache()
        net, criterion, optimizer = choose_net(dp_tec, image_size, batch_size)
        net.eval()
        net, hidden = train_net(net, trainloader, optimizer, criterion, appliance, dp_tec, image_size)
        save_net(net, appliance, dp_tec)

        #testing
        torch.cuda.empty_cache()
        print_start_test(appliance, dp_tec, 1, is_test=True)
        testloader = create_sequential_dataset(test, image_size, batch_size, target_cols)
        out = test_net(net, testloader, dp_tec, image_size, hidden=hidden)
        test = out_into_df(test, out, image_size)
        save_df_into_csv(test,   dp_tec, appliance, 1, is_test=True)

        # resultados em casas nao vistas
        for house in homes:
            torch.cuda.empty_cache()
            if house == 1:
                set_timewindow = True
                startDate = '2013-09-01'
                endDate = '2015-05-01'
            else:
                set_timewindow = False
            ukdale = DataSet(caminho_ukdale)
            if set_timewindow:
                ukdale.set_window(start=startDate, end=endDate)
            if house == 2 and appliance=='air dryer':
                pass
            df = create_dataset(ukdale, appliance, power, house)
            net.eval()

            print_start_test(appliance, dp_tec, house)
            dfloader = create_sequential_dataset(df, image_size, batch_size, target_cols)
            out = test_net(net, dfloader, dp_tec, image_size, hidden=hidden)
            df = out_into_df(df, out, image_size)
            save_df_into_csv(df, dp_tec, appliance, house)
        
    print('fim do aparelho ', appliance, 'as ', datetime.datetime.now())
    print('duracao de ', datetime.datetime.now() - comeco_aparelho)

comecou as  2022-10-26 12:35:19.060878
Comeco do aparelho  kettle  foi as  2022-10-26 12:35:19.062266
inicio treino: 2013-03-17 19:12:47
final treino: 2013-07-29 13:38:29
final teste: 2013-08-31 23:59:53


/tmp/ipykernel_7462/3504317227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ligado"] = df["ligado"].astype(int)


kettle  image_size = 48
###########################
Comeco do treino para  kettle  utilizando a rede  ANN  inicio as  2022-10-26 12:35:30.395741
[1,   100] loss: 0.344
[1,   200] loss: 0.280
[1,   300] loss: 0.288
[1,   400] loss: 0.208
[1,   500] loss: 0.174
[1,   600] loss: 0.150
[1,   700] loss: 0.154
[1,   800] loss: 0.172
[1,   900] loss: 0.135
[1,  1000] loss: 0.132
[2,   100] loss: 0.174
[2,   200] loss: 0.207
[2,   300] loss: 0.249
[2,   400] loss: 0.168
[2,   500] loss: 0.141
[2,   600] loss: 0.121
[2,   700] loss: 0.111
[2,   800] loss: 0.119
[2,   900] loss: 0.105
[2,  1000] loss: 0.106
[3,   100] loss: 0.124
[3,   200] loss: 0.153
[3,   300] loss: 0.210
[3,   400] loss: 0.158
[3,   500] loss: 0.110
[3,   600] loss: 0.111
[3,   700] loss: 0.099
[3,   800] loss: 0.111
[3,   900] loss: 0.094
[3,  1000] loss: 0.098
[4,   100] loss: 0.105
[4,   200] loss: 0.154
[4,   300] loss: 0.214
[4,   400] loss: 0.130
[4,   500] loss: 0.097
[4,   600] loss: 0.100
[4,   700] loss: 0.097
[4, 

/tmp/ipykernel_7462/721123282.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['saidas'] = 0.0
/tmp/ipykernel_7462/721123282.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, ('saidas')] = array/image_size


-----------------------------
comeco da validacao para casa 1 do  kettle utilizando  ANN foi as 2022-10-26 12:41:00.779372
Net(
  (hd1): Linear(in_features=240, out_features=2500, bias=True)
  (hd2): Linear(in_features=2500, out_features=2500, bias=True)
  (hd3): Linear(in_features=2500, out_features=2500, bias=True)
  (hd4): Linear(in_features=2500, out_features=48, bias=True)
  (bn): BatchNorm1d(2500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
10000
0:00:39.132424
20000
0:01:17.919754
30000
0:01:57.328716
40000
0:02:36.371035
50000
0:03:15.513114
60000
0:03:54.379253
70000
0:04:33.663420
80000
0:05:12.937255
90000
0:05:51.721678
100000
0:06:30.361551
110000
0:07:09.925209
duracao do teste: 0:07:11.848927
-----------------------------
-----------------------------
comeco da validacao para casa 2 do  kettle utilizando  ANN foi as 2022-10-26 12:48:58.229025
Net(
  (hd1): Linear(in_features=240, out_features=2500, bias=Tru

In [16]:
# for house in [2,5]:
#     torch.cuda.empty_cache()
#     ukdale = DataSet(caminho_ukdale)
#     set_timewindow = False
#     df = create_dataset(ukdale, appliance, power, house)
#     net.eval()

#     print_start_test(appliance, dp_tec, house)
#     dfloader = create_sequential_dataset(df, image_size, batch_size, target_cols)
#     out = test_net(net, dfloader, dp_tec, image_size, hidden=hidden)
#     df = out_into_df(df, out, image_size)
#     save_df_into_csv(df, dp_tec, appliance, house)

In [17]:
# # Testar thresholds e salvar metricas utilizadas

# for i in appliances:
#         app = appliances[i]['app']
#         for dp_tec in deep_learning_technique:
#                 print('Resultas para o teste do app:', app, 'usando:', dp_tec)
#                 test = open_csv_as_df(dp_tec, app, 1, is_test=True)
#                 text = try_thresholds(test)
#                 save_results_txt(text, app, dp_tec, house = 1, test = True)

#                 for house in homes:
#                         print('Resultas para a validacao do app:', app, 'usando:', dp_tec, 'na casa:', house)
#                         if house==5 and app=='hair dryer':
#                                 pass
#                         df = open_csv_as_df(dp_tec, app, house)
#                         text = try_thresholds(df)
#                         save_results_txt(text, app, dp_tec, house = house, test = False)

In [18]:
# import matplotlib
# from matplotlib import rcParams
# import matplotlib.pyplot as plt

# matplotlib.rc('font', size=MEDIUM_SIZE)
# matplotlib.rc('axes', titlesize=BIGGER_SIZE)

# threshold_df = pd.read_csv('resultados.csv', sep=',')

# def apply_threshold(df, threshold):
#     df = df.copy()
#     df.loc[:, 'ligado'] = df['ligado'].astype(int)
#     df.loc[:,'saidas'] = df['saidas'].apply(lambda x: x if x > 0 else 0)
#     df.loc[:,'saidas'] = df['saidas'].apply(lambda x: 1 if x > threshold else 0)
#     return df

# for i in appliances:
#         app = appliances[i]['app']
#         for dp_tec in deep_learning_technique:
#                 print('Resultas para o teste do app:', app, 'usando:', dp_tec)
#                 test = open_csv_as_df(dp_tec, app, 1, is_test=True)
#                 threshold = threshold_df[(threshold_df.Aparelho.str.lower() == app) & (threshold_df.Casa == 'Teste') &]['threshold']
#                 df = apply_threshold(test, threshold)
#                 plt.xlabel('Timestamp')
#                 plt.plot(test.index, df['ligado'], label = 'ligado')
#                 plt.plot(test.index, df['saidas'], label = "ligado_predicao")
#                 # plt.title(label='Resultados', fontdict= {'fontsize': rcParams['axes.titlesize'],
#                 # 'fontweight' : rcParams['axes.titleweight'],
#                 # 'verticalalignment': 'baseline',
#                 # 'horizontalalignment': 'center'})
#                 plt.ylabel('Resultado')
#                 plt.legend()
#                 plt.show()
#                 where_to_save = 'resultado_kettle_test.eps'
#                 plt.savefig(where_to_save, format='eps')



#                 for house in homes:
#                         print('Resultas para a validacao do app:', app, 'usando:', dp_tec, 'na casa:', house)
#                         if house==5 and app=='hair dryer':
#                                 pass
#                         df = open_csv_as_df(dp_tec, app, house)
#                         text = try_thresholds(df)
#                         save_results_txt(text, app, dp_tec, house = house, test = False)









In [19]:
# #se quiser pular o treino utilizando a rede do ANN
# path='BCEwithlogits_ANN_kettle_v2.pt'
# app = appliances[0]['app']
# dp_tec=deep_learning_technique[0]
# image_size = app_to_image_size[app]
# batch_size=64
# net = load_net(path, dp_tec, appliance, image_size, batch_size)

In [20]:
# import matplotlib
# from matplotlib import rcParams
# import matplotlib.pyplot as plt

# # matplotlib.rc('font', size=MEDIUM_SIZE)
# # matplotlib.rc('axes', titlesize=BIGGER_SIZE)

# threshold_df = pd.read_csv('resultados.csv', sep=',')

# def apply_threshold(df, threshold):
#     df = df.copy()
#     df.loc[:, 'ligado'] = df['ligado'].astype(int)
#     df.loc[:, 'saidas'] = df['saidas'].astype(int)
#     # df.loc[:,'saidas'] = df['saidas'].apply(lambda x: x if x > 0 else 0)
#     print(threshold)
#     df.loc[:,'saidas'] = df['saidas'].apply(lambda x: 1.02 if x > threshold else 0)
#     return df


# app = 'kettle'
# dp_tec = 'ANN'
# test = open_csv_as_df(dp_tec, app, 1, is_test=True)
# test = test.set_index('time')
# threshold = threshold_df[(threshold_df.Aparelho.str.lower() == app) & (threshold_df.Casa == 'Teste') & (threshold_df.Ferramenta == dp_tec)]['Threshold'][0]
# df = apply_threshold(test, threshold)
# plt.xlabel('Timestamp')
# plt.set_xticklabels(df,time)
# plt.plot(df.index, df['ligado'], label = 'ligado')
# plt.plot(df.index, df['saidas'], label = "ligado_predicao")
# # plt.title(label='Resultados', fontdict= {'fontsize': rcParams['axes.titlesize'],
# # 'fontweight' : rcParams['axes.titleweight'],
# # 'verticalalignment': 'baseline',
# # 'horizontalalignment': 'center'})
# plt.ylabel('Resultado')
# plt.legend()
# plt.show()
# where_to_save = 'resultado_kettle_test.eps'
# plt.savefig(where_to_save, format='eps')